In [ ]:
from pythonosc import dispatcher, osc_server
from pythonosc.udp_client import SimpleUDPClient
import biofeatures
import threading
import numpy as np

In [ ]:
def data_handler(unused_addr, args, data1, data2, data3, data4, data5, data6):
    global ecg_data, resp_data
    
    B = args[0]
    HR = args[1]
    
    # print(data5) # breathing
    # print(data6) # ECG
    
    ecg_data.append(float(data6))
    resp_data.append(float(data5))
        
    if B.is_warmed_up:
        B.set_data(resp_data[-B.buffer_length:])
    
    if HR.is_warmed_up:
        HR.set_data(ecg_data[-HR.buffer_length:])


In [ ]:
def warmup(B, resp_data, HR, ecg_data):
    """Function to launch once the system has warmed up.
    Sets the  data initially and launches a recursive update of features.
    Parameters
    ----------
    B: object containing breathing data and features
    resp_data: respiration signal data to set
    HR: object containing ECG data and features
    ecg_data: ecg signal data to set
    """
    
    if not B.is_warmed_up:
        print("Breathing WARMUP")
        B.set_data(resp_data[-B.buffer_length:])
        B.is_warmed_up = True
        B.update_loop()
        
    if not HR.is_warmed_up:
        print("ECG WARMUP")
        HR.set_data(ecg_data[-HR.buffer_length:])
        HR.is_warmed_up = True
        HR.update_loop()

In [ ]:
ip = "127.0.0.1"
port = 8000

ecg_data = []
resp_data = []

B = biofeatures.breathing(data = np.ones(10), buffer_length=1000)
HRV = biofeatures.hrv(data = np.ones(10), buffer_length=1000)

dispatcher = dispatcher.Dispatcher()
dispatcher.map("/0/bitalino", data_handler, B, HRV)

server = osc_server.ThreadingOSCUDPServer((ip, port), dispatcher)
print("Serving on {}".format(server.server_address))

# Warmup
warmup_t = 10
timer_warmup = threading.Timer(warmup_t, warmup, [B,resp_data,HRV,ecg_data])
timer_warmup.start()

try: 
    server.serve_forever()
except KeyboardInterrupt:
    B.update_data_flag = False
    HR.update_data_flag = False
except:
    raise

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.plot(ecg_data)